# Seminario de Ciencia de Datos

## 2º Cuatrimestre de 2017

## Módulo 2 - Series Temporales

### Integrantes:

- Mascitti, Julio Augusto - 954/11 - mascittija@gmail.com

- Aleman, Damián - ###/## - usuario@dominio

- Guerson, Matias Carlos - 925/10 - matias.guerson@gmail.com

#### Introducción

En este trabajo tenemos por objetivo trabajar con series temporlaes y grandes volúmenes de datos. Para lo mismo la catedra nos porporcionó un set de datos de Electroencefalogramas (EGGs) de 20 pacientes divididos en 2 grupos distinguibles. Estos datos fueron recolectados de mediciones realizadas por 256 electrodos ubicados en un casco que se conecta al paciente. El experimento conciste en estimular a los distintos pacientes conectados a los cascos para poder medir su reacción ante dicho estímulo.
Para más detalles leer el enunciado, que cuenta con información sobre la técnica de electroencefalografía y cuenta con detalles sobre la implementación del experimento.

### Analisis preliminar de los datos

In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
from scipy.stats import entropy
import scipy.io as io
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')

Importando los datos del primer paciente para familiarizarnos un poco con la información que disponemos

In [ ]:
# path = "../../../../"
path = "./"

data = io.loadmat('{}data/P01.mat'.format(path))

matriz = data['data']
print(matriz.shape)
print(len(matriz))   # x = epochs
print(len(matriz[0]))   # y = electrodos
print(len(matriz[0][0]))   # z = muestras 

Disponemos de una matriz por paciente con 894 Epochs, 256 electrodos y 201 muestras

In [ ]:
electrodo8 = matriz[:,7,:]
print(electrodo8.shape)
print(electrodo8)

In [ ]:
epoch_promedio = electrodo8.mean(axis=0)
plt.plot(epoch_promedio)

In [ ]:
frec, power = signal.welch(x= epoch_promedio,fs = 250)
print(frec)
plt.plot(frec, power)